<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd

import nltk

import re

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [2]:
try:
    df = pd.read_csv('toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
    
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
%%time
def clear_text(text):
    return " ".join(re.sub(r"[^a-zA-Z']", ' ', text).split()) 

df['clear_text'] = df['text'].apply(lambda x: clear_text(x))

CPU times: user 3.75 s, sys: 42.3 ms, total: 3.79 s
Wall time: 3.8 s


In [5]:
%%time

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemm_list = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    lemm_text = " ".join(lemm_list)
    return lemm_text

#Лемматизация
df['lemm_text'] = df['clear_text'].apply(lambda x: lemmatize(x))

CPU times: user 16min 7s, sys: 1min 51s, total: 17min 58s
Wall time: 17min 59s


In [6]:
%%time
stopwords = set(nltk_stopwords.words('english'))
#count_tf_idf = TfidfVectorizer(min_df=0.1, sublinear_tf=True, ngram_range = (1,2), stop_words=stopwords)
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

CPU times: user 2.4 ms, sys: 0 ns, total: 2.4 ms
Wall time: 4.82 ms


In [7]:
#Было
#features = count_tf_idf.fit_transform(df['lemm_text'].values) 
#target = df['toxic']

#Нужно обучать TfidfVectorizer на тренировочной выборке, после чего трансформировать тренировочную и тестовую выборку
#Стало
text_train, text_test, target_train, target_test = (
    train_test_split(df['lemm_text'].values, df['toxic'], test_size=0.2, random_state=12345)
)

count_tf_idf.fit(text_train)
features_train = count_tf_idf.transform(text_train)
features_test = count_tf_idf.transform(text_test)

features_train.shape, features_test.shape

((127433, 139501), (31859, 139501))

## Обучение

In [8]:
%%time


# parameter grid
parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-3,3,7),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],
}
clf = GridSearchCV(LogisticRegression(),                    # model
                   param_grid = parameters,   # hyperparameters
                   scoring='f1',        # metric for scoring
                   cv=10)                     # number of folds
clf.fit(features_train, target_train)
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)
model = clf.best_estimator_


#model = LogisticRegression(random_state=12345, class_weight='balanced', max_iter = 300, C=0.9)
#model.fit(features_train, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610

Tuned Hyperparameters : {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
Accuracy : 0.7635346417368162
CPU times: user 35min 41s, sys: 52min, total: 1h 27min 42s
Wall time: 1h 27min 47s


In [9]:
pred = model.predict(features_train)
print('F1 на обучающей выборке: {:.3f}'.format(f1_score(target_train, pred)))

F1 на обучающей выборке: 0.961


## Выводы

In [10]:
#Проверка на тестовой выборке
pred = model.predict(features_test)
print('F1 на тестовой выборке: {:.3f}'.format(f1_score(target_test, pred)))

F1 на тестовой выборке: 0.770


Проведен подбор и обучение модели логистической регрессии с помощью TF-IDF, которая умеет классифицировать тексты комментариев на токсичные и не являющиеся таковыми. Метрика F1 на обучающей выборке составила 0.961, на тестовой - 0.770. Порог пройдет.